# 1. Retrieval Augmented In-context Learning

- Please choose "Runtime Type" = GPU in Colab for running this notebook (Runtime > Change Runtime Type > T4 GPU).
- You are free to choose to use Google Colab or Kaggle to complete this notebook.

## 1.1 Contextual Embedding

In [6]:
# Step 0. Prepare the environment
!pip install InstructorEmbedding sentence-transformers datasets scikit-learn

  Using cached InstructorEmbedding-1.0.1-py2.py3-none-any.whl (19 kB)
  Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
  Using cached datasets-2.19.0-py3-none-any.whl (542 kB)
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

In [7]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/test-blind.txt

--2024-04-29 09:16:04--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 738844 (722K) [text/plain]
Saving to: ‘data/classification/train.txt’

data/classification 100%[===================>] 721.53K  --.-KB/s    in 0.03s   

2024-04-29 09:16:04 (25.0 MB/s) - ‘data/classification/train.txt’ saved [738844/738844]

--2024-04-29 09:16:04--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [8]:
# Step 1. Declare the model & Example usage
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("hkunlp/instructor-base")
embeddings = model.encode(
    [
        "Dynamical Scalar Degree of Freedom in Horava-Lifshitz Gravity",
        "Comparison of Atmospheric Neutrino Flux Calculations at Low Energies",
        "Fermion Bags in the Massive Gross-Neveu Model",
        "QCD corrections to Associated t-tbar-H production at the Tevatron",
    ],
    prompt="Represent the Medicine sentence for clustering: ",
    show_progress_bar=True,
)

print(embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

2_Dense/pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(4, 768)


In [9]:
# Step 2. Load data & Extract embeddings
# It may takes about two hours for CPU, or you can switch to GPU runtime for faster inference.
def load_data(file):
    r"""Load your custom data for training or evaluation.

    Args:
        file (str): the path of your data

    Returns:
        tuple: a tuple containing two elements:
           - embedding: The embedding of your data, should in shape (N, 768)
          - label: List of labels, should in shape (N,)
    """
    with open(file) as f:
        data = [line.strip().split('\t') for line in f]
    sentences = [line[1] for line in data]
    X = model.encode(sentences, prompt="Represent the sentence for sentiment analysis: ", show_progress_bar=True)
    y = [int(line[0]) for line in data]
    return X, y

# x: embedding; y: label
X_train, y_train = load_data('data/classification/train.txt')
X_val, y_val = load_data('data/classification/dev.txt')

print(f"X_train shape: {X_train.shape}, y_train shape: {len(y_train)}")
print(f"X_val shape: {X_val.shape}, y_val shape: {len(y_val)}")

Batches:   0%|          | 0/217 [00:00<?, ?it/s]

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

X_train shape: (6920, 768), y_train shape: 6920
X_val shape: (872, 768), y_val shape: 872


And train a logistic regression model

In [10]:
# Step 3: Train a logistic regression mode
# • Use the same logistic regression method used in Assignment 1 to predict the sentiment labels of
# movie reviews. As before, only use the "train.txt" file to train the model.
from sklearn.linear_model import LogisticRegression

# Fit the LogisticRegression model
logReg = LogisticRegression()
logReg.fit(X_train, y_train)

LogisticRegression()

In [11]:
# Step 4: Evaluate the model
# Tabulate and report the precision (P), recall (R), and F1 scores from the "dev.txt" file
# and compare this performance with the GloVe features
from sklearn.metrics import precision_score, recall_score, f1_score

predictions = logReg.predict(X_val)

print("Instructor-base precision: ", precision_score(y_val, predictions, average='macro'))
print("Instructor-base recall: ", recall_score(y_val, predictions, average='macro'))
print("Instructor-base f1 score: ", f1_score(y_val, predictions, average='macro'))

Instructor-base precision:  0.9051861807257491
Instructor-base recall:  0.9045529174033847
Instructor-base f1 score:  0.9047251712948642


## 1.2 Retrieve Relevant Examples

In [12]:
import re
import json
from datasets import load_dataset
from sentence_transformers import SentenceTransformer

def load_train_data():
    """Loads the GSM8k train dataset.

    Returns:
        A list of dictionaries containing questions, cot answers, and short answers.
    """
    ds = load_dataset("gsm8k", "main", split="train")
    examples = [{"question": example["question"], "answer": example["answer"]} for example in ds]
    for example in examples:
        example["short_answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"].split("####")[1].strip())
        example["cot_answer"] = re.sub(r"\<\<.*?\>\>", "", example["answer"].split("####")[0].strip()) \
            + " So the answer is " + example["short_answer"] + "."
    return examples

def load_test_data():
    """Loads the first 30 examples of the GSM8k test dataset.

    Returns:
        A list of dictionaries containing questions and answers.
    """
    ds = load_dataset("gsm8k", "main", split="test")
    examples = [{"question": example["question"], "answer": example["answer"].split("####")[1].strip()} for example in ds]
    for example in examples:
        example["answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"])
    return examples[:30]

In [13]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def retrieve_examples():
    """Retrieve top-20 in-context examples from GSM8K training set for each testing examples.

    Returns:
        A dictionary mapping testing questions to a list of top-20 training examples.
    """
    examples = load_train_data()        # Returns: A list of dictionaries containing questions, cot answers, and short answers.
    test_data = load_test_data()        # Returns: A list of dictionaries containing questions and answers.

    # Convert the examples and test data to a format suitable for TF-IDF Vectorizer
    train_questions = [example['question'] for example in examples]
    test_questions = [test['question'] for test in test_data]

    # Initialize the TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the training questions and transform the test questions
    train_vectors = vectorizer.fit_transform(train_questions)
    test_vectors = vectorizer.transform(test_questions)

    # Calculate the cosine similarity between the test vectors and the train vectors
    cosine_similarities = cosine_similarity(test_vectors, train_vectors)

    # For each test example, get the top 20 training examples with the highest cosine similarity
    top_20_examples = {}
    for i, test_question in enumerate(test_questions):
        top_20_indices = np.argsort(cosine_similarities[i])[-20:]
        top_20_examples[test_question] = [examples[j] for j in top_20_indices]

    return top_20_examples




    # train = load_train_data()
    # test = load_test_data()

    # test_ques = {}
    # cos_sim = {}
    # top20 = []

    # for example in train:
    #     test_ques[example["question"]] = []
    #     for test_example in test:
    #         cos_sim[example["question"]] = cosine_similarity(model.encode([example["question"]]), model.encode([test_example["question"]]))[0][0]

    # # sort
    # cos_sim = dict(sorted(cos_sim.items(), key=lambda item: item[1], reverse=True))

    # for i in range(20):
    #     top20.append(list(cos_sim.keys())[i])

    # return top20


    # raise NotImplmentedError()

In [14]:
RETRIVED_EXAMPLES = retrieve_examples()

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Note: The following retrieval augmented generation does not require a GPU. Please consider saving and downloading the examples you retrieve from the left file tab so that you will not be hindered by Colab GPU restrictions.

In [15]:
with open("retrieved_examples.json", "w") as fout:
    json.dump(RETRIVED_EXAMPLES, fout)

with open("retrieved_examples.json", "r") as fin:
    RETRIVED_EXAMPLES = json.load(fin)

## 1.3 Generation with Huggingface Inference API

We will use LLM by querying huggingface inference api so we do not need GPU for the following code. Please generate HF_TOKEN at [hf.co/settings/tokens](hf.co/settings/tokens) and set as environment varible.

In [16]:
!pip install backoff evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [30]:
from abc import ABC, abstractmethod
from typing import List, Dict, Any
import os
import json
import backoff
import evaluate
import re
import time
import requests


os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_TOKEN"] = "<YOUR TOKEN>"

from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

class LLM(object):
    def __init__(self, model_name="codellama/CodeLlama-7b-hf"):
        self.model_name = model_name
        self.api_url = f"https://api-inference.huggingface.co/models/{model_name}"
        self.headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}

    @backoff.on_exception(backoff.expo, requests.exceptions.RequestException, max_time=60)
    def generate(self, prompts: List[str], **kwargs) -> List[str]:
        outputs = []

        def query(payload):
            response = requests.post(self.api_url, headers=self.headers, json=payload)
            if response.status_code != 200:
                raise ValueError(f"Request failed with code {response.status_code}, {response.text}")
            return response.json()

        for prompt in prompts:
            data = query(
                {
                    "inputs": prompt,
                    "parameters": { "max_new_tokens": 256, "stop": ["Question:"]},
                }
            )

            outputs.append(data[0]['generated_text'])

        return outputs

In [31]:
llm = LLM("codellama/CodeLlama-7b-hf")
# print(llm.generate(["Explain the importance of low latency LLMs", "What is the capital of France?"]))

Please adapt your GSM8KCoTEvaluator for this API-based LLM. And report the performance of 8-shot chain-of-thought prompting on first 30 examples of GSM8K.

In [32]:
class Evaluator(ABC):
    def __init__(self, llm):
        self.llm = llm

    @abstractmethod
    def load_data(self):
        pass

    @abstractmethod
    def build_prompts(self):
        pass

    @abstractmethod
    def postprocess_output(self, output: str) -> str:
        pass

    def generate_completions(self, prompts: List[str], batch_size=2) -> List[str]:
        generated_completions = []

        for i in range(0, len(prompts), batch_size):
          batch_prompts = prompts[i:i+batch_size]
          batch_id = self.llm.generate(batch_prompts)
          generated_completions.append(batch_id)
          # print(f"{i}: {batch_id}")
          # print(batch_id)
          # for j in batch_id:
          #   generated_completions.append([self.llm.tokenizer.batch_decode(batch_id)])
        # print(f"generated_completions: {generated_completions}")
        return generated_completions
        pass

    def evaluate(self, batch_size=4, n_shot=8, save_dir="outputs"):
        dataset = self.load_data()
        prompts = self.build_prompts(dataset, n_shot)
        outputs = self.generate_completions(prompts, batch_size=batch_size)

        predictions = []
        # print(f"dataset: {dataset}")
        # print(f"prompts: {prompts}")
        # print(f"outputs: {outputs}")
        # print(len(outputs))
        for i, (example, prompt, output) in enumerate(zip(dataset, prompts, outputs)):
            prediction = {
                "task_id": example.get("task_id", f"task_{i}"),
                "prompt": prompt,
                "completion": output,
                "prediction": self.postprocess_output(output),
            }
            predictions.append(prediction)

        # Save predictions to file
        os.makedirs(save_dir, exist_ok=True)
        prediction_save_path = os.path.join(save_dir, f"{type(self).__name__}_predictions.jsonl")
        with open(prediction_save_path, "w") as fout:
            for pred in predictions:
                fout.write(json.dumps(pred) + "\n")

        # Calculate metrics and print results
        results = self.calculate_metrics(predictions, dataset)
        print(f"Results for {type(self).__name__}: {results}")

    @abstractmethod
    def calculate_metrics(self):
        pass

GSM_EXAMPLARS = [
    {
        "question": "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
        "cot_answer": "There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. So the answer is 6.",
        "pot_answer": "def solution():\n    \"\"\"There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?\"\"\"\n    trees_initial = 15\n    trees_after = 21\n    trees_added = trees_after - trees_initial\n    result = trees_added\n    return result",
        "short_answer": "6"
    },
    {
        "question": "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
        "cot_answer": "There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. So the answer is 5.",
        "pot_answer": "def solution():\n    \"\"\"If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?\"\"\"\n    cars_initial = 3\n    cars_arrived = 2\n    total_cars = cars_initial + cars_arrived\n    result = total_cars\n    return result",
        "short_answer": "5"
    },
    {
        "question": "Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?",
        "cot_answer": "Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. So the answer is 39.",
        "pot_answer": "def solution():\n    \"\"\"Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?\"\"\"\n    leah_chocolates = 32\n    sister_chocolates = 42\n    total_chocolates = leah_chocolates + sister_chocolates\n    chocolates_eaten = 35\n    chocolates_left = total_chocolates - chocolates_eaten\n    result = chocolates_left\n    return result",
        "short_answer": "39"
    },
    {
        "question": "Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?",
        "cot_answer": "Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. So the answer is 8.",
        "pot_answer": "def solution():\n    \"\"\"Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?\"\"\"\n    jason_lollipops_initial = 20\n    jason_lollipops_after = 12\n    denny_lollipops = jason_lollipops_initial - jason_lollipops_after\n    result = denny_lollipops\n    return result",
        "short_answer": "8"
    },
    {
        "question": "Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?",
        "cot_answer": "Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. So the answer is 9.",
        "pot_answer": "def solution():\n    \"\"\"Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\"\"\"\n    toys_initial = 5\n    mom_toys = 2\n    dad_toys = 2\n    total_received = mom_toys + dad_toys\n    total_toys = toys_initial + total_received\n    result = total_toys\n    return result",
        "short_answer": "9"
    },
    {
        "question": "There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?",
        "cot_answer": "There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. So the answer is 29.",
        "pot_answer": "def solution():\n    \"\"\"Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?\"\"\"\n    toys_initial = 5\n    mom_toys = 2\n    dad_toys = 2\n    total_received = mom_toys + dad_toys\n    total_toys = toys_initial + total_received\n    result = total_toys\n    return result",
        "short_answer": "29"
    },
    {
        "question": "Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?",
        "cot_answer": "Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. So the answer is 33.",
        "pot_answer": "def solution():\n    \"\"\"Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?\"\"\"\n    golf_balls_initial = 58\n    golf_balls_lost_tuesday = 23\n    golf_balls_lost_wednesday = 2\n    golf_balls_left = golf_balls_initial - golf_balls_lost_tuesday - golf_balls_lost_wednesday\n    result = golf_balls_left\n    return result",
        "short_answer": "33"
    },
    {
        "question": "Olivia has $23. She bought five bagels for $3 each. How much money does she have left?",
        "cot_answer": "Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8. So the answer is 8.",
        "pot_answer": "def solution():\n    \"\"\"Olivia has $23. She bought five bagels for $3 each. How much money does she have left?\"\"\"\n    money_initial = 23\n    bagels = 5\n    bagel_cost = 3\n    money_spent = bagels * bagel_cost\n    money_left = money_initial - money_spent\n    result = money_left\n    return result",
        "short_answer": "8"
    }
]

In [33]:
class GSM8KEvaluator(Evaluator):
    def load_data(self):
        ds = load_dataset("gsm8k", "main", split="test")
        examples = [{"question": example["question"], "answer": example["answer"].split("####")[1].strip()} for example in ds]
        for example in examples:
            example["answer"] = re.sub(r"(\d),(\d)", r"\1\2", example["answer"])
        return examples[:30]

    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        pass

    def postprocess_output(self, output: str) -> str:
        stop_sequences=["\nclass", "\ndef", "\n#", "\nif", "\nprint"]
        # print(f"pre: {output}")
        # for seq in stop_sequences:
        #     if seq in output:
        #         output = output.split(seq)[0]
        #         break

        # ans = output.split("Question:")[-1]
        # print(f"post processed output :{ans}")
        # print(f"post processed output :{output}")
        # return output.strip()
        return output
        # pass

    def calculate_metrics(self, predictions, dataset):
        correctCnt = 0
        # for pred, example in zip(predictions, dataset):
        for example in dataset:
            pred_ans = ""
            # p = pred["prediction"]
            # print(f"pred: {p}")
            # print(f"pred: {p}")
            # print("example ques: ", example["question"])
            for pred in predictions:
              # print("pred: ", pred["prediction"])
              for p in pred["prediction"]:

                if(example["question"] in p):
                  # print("p: ", p)
                  # print("p split: ", p.split("\nQuestion:")[-2])
                  pred_ans = p.split(".\nQuestion:")[-2].split(" ")
                  if(pred_ans[-1].replace(".", "").isnumeric()):
                    pred_ans = pred_ans[-1]
                  else:
                    for i in range(len(pred_ans)-1, 0, -1):
                      if(pred_ans[i].replace(".", "").isnumeric()):
                        pred_ans = pred_ans[i]
                        break
                  # print("example ans: ", example["answer"])
                  # print("pred_ans: ", pred_ans)
            # print(f"example: {example}")


            if pred_ans == example["answer"]:
                correctCnt += 1
                # print("pred_ans ", pred_ans, " example['answer'] ", example["answer"])

        # print("len(dataset) ", len(dataset), "len(predictions) ", len(predictions))
        return correctCnt / len(dataset)
        # return correctCnt / len(predictions)
        # return {"accuracy": correctCnt / len(predictions)}
        # pass

In [34]:
class GSM8KCoTEvaluator(GSM8KEvaluator):
    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        prompts = []
        prompt = "Answer the following questions.\n"
        count = 0
        for demo in demos:
            if(count >= n_shot):
              break
            prompt += "Question: " + demo["question"] + "\n"
            prompt += "Answer: " + demo["cot_answer"] + "\n"
            count+=1

        for example in dataset:
            prompts.append(prompt + "Question: " + example["question"] + "\n")

        # print(f"num of prompts: {len(prompts)}")
        return prompts
        # pass

In [35]:
cot_evaluator = GSM8KCoTEvaluator(llm)
cot_evaluator.evaluate(n_shot=8)

Results for GSM8KCoTEvaluator: 0.1


## 1.4 Impact of Quantity on Few-shot Prompting

In [36]:
cot_evaluator.evaluate(n_shot=1)
cot_evaluator.evaluate(n_shot=2)
cot_evaluator.evaluate(n_shot=4)

Results for GSM8KCoTEvaluator: 0.06666666666666667
Results for GSM8KCoTEvaluator: 0.06666666666666667
Results for GSM8KCoTEvaluator: 0.1


## 1.5 Retrieval Augmented Few-shot Prompting

In [22]:
class GSM8KRetrievalICLEvaluator(GSM8KEvaluator):
    def build_prompts(self, dataset, n_shot=8, demos=GSM_EXAMPLARS):
        """Build prompts with RETRIVED_EXAMPLES we generated in 1.2.
        """
        # print(dataset)

        # for demo in RETRIVED_EXAMPLES:
        #     if(count >= n_shot):
        #       break
        #     prompt += "Question: " + demo["question"] + "\n"
        #     prompt += "Answer: " + demo["cot_answer"] + "\n"
        #     count+=1
        prompts = []

        for example in dataset:
          prompt = "Answer the following questions.\n"
          count = 0
          for demo in RETRIVED_EXAMPLES[example["question"]]:
            if(count >= n_shot):
              break
            prompt += "Question: " + demo["question"] + "\n"
            prompt += "Answer: " + demo["cot_answer"] + "\n"
            count+=1


          prompts.append(prompt + "Question: " + example["question"] + "\n")

        # print(f"num of prompts: {len(prompts)}")
        return prompts

In [23]:
retrieval_icl_evaluator = GSM8KRetrievalICLEvaluator(llm)
# retrieval_icl_evaluator.evaluate(n_shot=8)

In [24]:
# retrieval_icl_evaluator = GSM8KRetrievalICLEvaluator(llm)
retrieval_icl_evaluator.evaluate(n_shot=1)
retrieval_icl_evaluator.evaluate(n_shot=2)
retrieval_icl_evaluator.evaluate(n_shot=4)
retrieval_icl_evaluator.evaluate(n_shot=8)

Results for GSM8KRetrievalICLEvaluator: 0.03333333333333333
Results for GSM8KRetrievalICLEvaluator: 0.06666666666666667
Results for GSM8KRetrievalICLEvaluator: 0.03333333333333333
Results for GSM8KRetrievalICLEvaluator: 0.16666666666666666


In [25]:
# Rearrange the examples in ascending order of relevance.
for example in RETRIVED_EXAMPLES:
  RETRIVED_EXAMPLES[example] = RETRIVED_EXAMPLES[example][::-1]

retrieval_icl_evaluator.evaluate(n_shot=1)
retrieval_icl_evaluator.evaluate(n_shot=2)
retrieval_icl_evaluator.evaluate(n_shot=4)
retrieval_icl_evaluator.evaluate(n_shot=8)

Results for GSM8KRetrievalICLEvaluator: 0.03333333333333333
Results for GSM8KRetrievalICLEvaluator: 0.13333333333333333
Results for GSM8KRetrievalICLEvaluator: 0.2
Results for GSM8KRetrievalICLEvaluator: 0.1


## Discussion

- Q1.1: Regarding the contextual embedding in Section 1.1, how does sentiment classification performance compare to your word2vec results in A1? Discuss potential reasons.

GloVe precision:  0.7668189895912668

GloVe recall:  0.7654710785551907

GloVe f1 score:  0.7655617408906883

Instructor-base precision:  0.9051861807257491

Instructor-base recall:  0.9045529174033847

Instructor-base f1 score:  0.9047251712948642

All scores (precision, recall, f1) are higher using Instructor-base than Glove.

- Q1.2: In Section 1.4, which discusses the impact of quantity, what trends do you notice when adding contextual examples? Do you think this trend will continue?

the accuracy goes up when there are more contextual examples. this trend is likely to continue since more context would help the understanding of the task which often yields better results.

- Q1.3: In Section 1.5, which covers retrieval-augmented in-context learning, how does this differ from Section 1.4? Analyze the reasons.

in section 1.5, only the examples that are relavant to the question to be asked are included in the prompt



- Q1.4: In Section 1.5, which arrangement yields better performance: in-context examples organized in descending or ascending order of relevance? Discuss the scenario.

ascending order gives better performance (ie top1, top2, ..., top8)



# 2. Basic Decoding Algorithms

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
"""set device and random seeds"""

######################################################
#  The following helper functions are given to you.
######################################################

from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

def set_seed(seed=19260817):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

device: cuda


In [ ]:
"""load datasets"""

######################################################
#  The following helper code is given to you.
######################################################

from datasets import load_dataset

dataset = load_dataset('Ximing/ROCStories')
train_data, dev_data, test_data = dataset['train'], dataset['validation'], dataset['test']

print(train_data[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16207 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1817 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1819 [00:00<?, ? examples/s]

{'story_id': '080198fc-d0e7-42b3-8e63-b2144e59d816', 'prompt': 'On my way to work I stopped to get some coffee.', 'continuation': 'I went through the drive through and placed my order. I paid the cashier and patiently waited for my drink. When she handed me the drink, the lid came off and spilled on me. The coffee hurt and I had to go home and change clothes.', 'constraint_words': ['drive', 'order', 'drink', 'lid', 'coffee', 'hurt', 'home', 'change', 'clothes']}


In [ ]:
"""prepare evaluation"""

######################################################
#  The following helper code is given to you.
######################################################

from evaluate import load
from transformers import RobertaForSequenceClassification, RobertaTokenizer

perplexity_scorer = load("perplexity", module_type="metric")
cola_model_name = "textattack/roberta-base-CoLA"
cola_tokenizer = RobertaTokenizer.from_pretrained(cola_model_name)
cola_model = RobertaForSequenceClassification.from_pretrained(cola_model_name).to(device)

def batchify(data, batch_size):
    assert batch_size > 0

    batch = []
    for item in data:
        # Yield next batch
        if len(batch) == batch_size:
            yield batch
            batch = []

        batch.append(item)

    # Yield last un-filled batch
    if len(batch) != 0:
        yield batch

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
"""set up evaluation metric"""

######################################################
#  The following helper code is given to you.
######################################################

def compute_perplexity(texts, model='gpt2', batch_size=8):
    score = perplexity_scorer.compute(predictions=texts, add_start_token=True, batch_size=batch_size, model_id=model)
    return score['mean_perplexity']


def compute_fluency(texts, batch_size=8):
  scores = []
  for b_texts in batchify(texts, batch_size):
    inputs = cola_tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
      logits = cola_model(**inputs).logits
      probs = logits.softmax(dim=-1)
      scores.extend(probs[:, 1].tolist())
  return sum(scores) / len(scores)


def compute_diversity(texts):
    unigrams, bigrams, trigrams = [], [], []
    total_words = 0
    for gen in texts:
        o = gen.split(' ')
        total_words += len(o)
        for i in range(len(o)):
            unigrams.append(o[i])
        for i in range(len(o) - 1):
            bigrams.append(o[i] + '_' + o[i + 1])
        for i in range(len(o) - 2):
            trigrams.append(o[i] + '_' + o[i + 1] + '_' + o[i + 2])
    return len(set(unigrams)) / len(unigrams), len(set(bigrams)) / len(bigrams), len(set(trigrams)) / len(trigrams)


def evaluate(generations, experiment):
  generations = [_ for _ in generations if _ != '']
  perplexity = compute_perplexity(generations)
  fluency = compute_fluency(generations)
  diversity = compute_diversity(generations)
  print(experiment)
  print(f'perplexity = {perplexity:.2f}')
  print(f'fluency = {fluency:.2f}')
  print(f'diversity = {diversity[0]:.2f}, {diversity[1]:.2f}, {diversity[2]:.2f}')
  print()

debug_sents = ["This restaurant is awesome", "My dog is cute and I love it.", "Today is sunny."]
evaluate(debug_sents, 'debugging run')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

debugging run
perplexity = 178.64
fluency = 0.98
diversity = 0.87, 1.00, 1.00



In [ ]:
"""load model and tokenizer"""

######################################################
#  The following helper code is given to you.
######################################################

from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name, pad_token="<|endoftext|>")
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In this section, you will implement a few basic decoding algorithms:
1. Greedy decoding
2. Vanilla sampling
3. Temperature sampling
5. Top-p sampling

We have provided a wrapper function `decode()` that takes care of batching, controlling max length, and handling the EOS token.
You will be asked to implement the core function of each method: *given the pre-softmax logits of the next token, decide what the next token is.*

**The wrapper calls the core function of each decoding algorithm, which you will implement in the subsections below.**

In [ ]:
"""decode main wrapper function"""

######################################################
#  The following helper code is given to you.
######################################################

def decode(prompts, max_len, method, **kwargs):
  encodings_dict = tokenizer(prompts, return_tensors="pt", padding=True)
  input_ids = encodings_dict['input_ids'].to(device)
  attention_mask = encodings_dict['attention_mask'].to(device)

  model_kwargs = {'attention_mask': attention_mask}
  batch_size, input_seq_len = input_ids.shape

  unfinished_sequences = torch.ones(batch_size, dtype=torch.long, device=device)

  for step in range(max_len):
    model_inputs = model.prepare_inputs_for_generation(input_ids, **model_kwargs)
    with torch.no_grad():
      outputs = model(**model_inputs, return_dict=True, output_attentions=False, output_hidden_states=False)

    if step == 0:
      last_non_masked_idx = torch.sum(attention_mask, dim=1) - 1
      next_token_logits = outputs.logits[range(batch_size), last_non_masked_idx, :]
    else:
      next_token_logits = outputs.logits[:, -1, :]

    log_prob = F.log_softmax(next_token_logits, dim=-1)

    if method == 'greedy':
      next_tokens = greedy(next_token_logits)
    elif method == 'sample':
      next_tokens = sample(next_token_logits)
    elif method == 'temperature':
      next_tokens = temperature(next_token_logits, t=kwargs.get('t', 0.8))
    elif method == 'topk':
      next_tokens = topk(next_token_logits, k=kwargs.get('k', 20))
    elif method == 'topp':
      next_tokens = topp(next_token_logits, p=kwargs.get('p', 0.7))

    # finished sentences should have their next token be a padding token
    next_tokens = next_tokens * unfinished_sequences + tokenizer.pad_token_id * (1 - unfinished_sequences)

    input_ids = torch.cat([input_ids, next_tokens[:, None]], dim=-1)
    model_kwargs = model._update_model_kwargs_for_generation(outputs, model_kwargs, is_encoder_decoder=model.config.is_encoder_decoder)

    # if eos_token was found in one sentence, set sentence to finished
    unfinished_sequences = unfinished_sequences.mul((next_tokens != tokenizer.eos_token_id).long())

    if unfinished_sequences.max() == 0:
      break

  response_ids = input_ids[:, input_seq_len:]
  response_text = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in response_ids]

  return response_text

In [ ]:
"""debug helper code"""

######################################################
#  The following helper code is given to you.
######################################################

# For debugging, we duplicate a single prompt 10 times so that we obtain 10 generations for the same prompt
dev_prompts = [dev_data[0]['prompt']] * 10

def print_generations(prompts, generations):
  for prompt, generation in zip(prompts, generations):
    print(f'{[prompt]} ==> {[generation]}')

## 2.1 Greedy Decoding

In [ ]:
def greedy(next_token_logits):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # TODO: compute `next_tokens` from `next_token_logits`.
  # given the pre-softmax logits of the next token, decide what the next token is.
  # Hint: use torch.argmax()
  next_tokens = torch.argmax(next_token_logits, dim=-1)

  return next_tokens


generations = decode(dev_prompts, max_len=20, method='greedy')
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m goin

## 2.2 Vanilla Sampling and Temperature Sampling

In [ ]:
def sample(next_token_logits):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # TODO: compute the probabilities `probs` from the logits.
  # Hint: `probs` should have size (B, V)
  probs = F.softmax(next_token_logits, dim=-1)

  # TODO: compute `next_tokens` from `probs`.
  # Hint: use torch.multinomial()
  next_tokens = torch.multinomial(probs, 1).squeeze()

  return next_tokens


set_seed()
generations = decode(dev_prompts, max_len=20, method='sample')
print_generations(dev_prompts, generations)

['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"Let\'s walk through the window," the banner states (it didn\'t say that at']
['Ryan was called by his friend to skip work one day.'] ==> [' Tucked inside a basement window, he found a packed pack of cigarettes on a tin counter in the']
['Ryan was called by his friend to skip work one day.'] ==> [' He said he wanted to enroll in school next Thursday.\n\nPolice said a woman and her child']
['Ryan was called by his friend to skip work one day.'] ==> [' The person had taken him on vacation. And now a passenger jumped in the cab, snatched her purse']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\nInvestigators remain in intensive care at the hospital, where they will continue to examine memories and possible']
['Ryan was called by his friend to skip work one day.'] ==> [' "Half of us left to go into vision, so we did it," said lawyer Douglas Klam']
['Ryan was called by his friend to skip work one day.'] ==

In [ ]:
def temperature(next_token_logits, t):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  - t: float
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # TODO: compute the probabilities `probs` from the logits, with temperature applied.
  probs = F.softmax(next_token_logits / t, dim=-1)

  # TODO: compute `next_tokens` from `probs`.
  next_tokens = torch.multinomial(probs, 1).squeeze()

  return next_tokens


set_seed()
# generations = decode(dev_prompts, max_len=20, method='temperature', t=0.8)
# print_generations(dev_prompts, generations)
print("t = 1")
generations = decode(dev_prompts, max_len=20, method='temperature', t=1)
print_generations(dev_prompts, generations)
print("t = 0.001")
generations = decode(dev_prompts, max_len=20, method='temperature', t=0.001)
print_generations(dev_prompts, generations)

t = 1
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"Let\'s walk through the window," the banner states (it didn\'t say that at']
['Ryan was called by his friend to skip work one day.'] ==> [' Tucked inside a basement window, he found a packed pack of cigarettes on a tin counter in the']
['Ryan was called by his friend to skip work one day.'] ==> [' He said he wanted to enroll in school next Thursday.\n\nPolice said a woman and her child']
['Ryan was called by his friend to skip work one day.'] ==> [' The person had taken him on vacation. And now a passenger jumped in the cab, snatched her purse']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\nInvestigators remain in intensive care at the hospital, where they will continue to examine memories and possible']
['Ryan was called by his friend to skip work one day.'] ==> [' "Half of us left to go into vision, so we did it," said lawyer Douglas Klam']
['Ryan was called by his friend to skip work one day

## 2.3 Top-p Sampling

In [ ]:
def topp(next_token_logits, p):
  '''
  inputs:
  - next_token_logits: Tensor(size = (B, V), dtype = float)
  - p: float
  outputs:
  - next_tokens: Tensor(size = (B), dtype = long)
  '''

  # TODO: Sort the logits in descending order, and compute
  # the cumulative probabilities `cum_probs` on the sorted logits
  sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True, dim=-1)
  sorted_probs = F.softmax(sorted_logits, dim=-1)
  cum_probs = torch.cumsum(sorted_probs, dim=-1)

  # Create a mask to zero out all logits not in top-p
  sorted_indices_to_remove = cum_probs > p
  sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
  sorted_indices_to_remove[:, 0] = 0
  # Restore mask to original indices
  indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)

  # Mask the logits
  next_token_logits[indices_to_remove] = float('-inf')

  # TODO: Sample from the masked logits
  probs = F.softmax(next_token_logits, dim=-1)
  next_tokens = torch.multinomial(probs, 1).squeeze()

  return next_tokens


set_seed()
# generations = decode(dev_prompts, max_len=20, method='topp', p=0.7)
# print_generations(dev_prompts, generations)
print("p = 0")
generations = decode(dev_prompts, max_len=20, method='topp', p=0)
print_generations(dev_prompts, generations)
print("p = 0.99")
generations = decode(dev_prompts, max_len=20, method='topp', p=0.99)
print_generations(dev_prompts, generations)

p = 0
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'m going to go to work tomorrow,\'" he said.']
['Ryan was called by his friend to skip work one day.'] ==> ['\n\n"I was like, \'I\'

## 2.4: Evaluation

Run the following cell to obtain the evaluation results, which you should include in your writeup.
Also don't forget to answer the questions.

In [ ]:
prompts = [item['prompt'] for item in test_data][:10]
GENERATIONS_PER_PROMPT = 10
MAX_LEN = 100

for experiment in ['greedy', 'sample', 'temperature', 'topp']:
  generations = []
  for prompt in tqdm(prompts):
    generations += decode([prompt] * GENERATIONS_PER_PROMPT, max_len=MAX_LEN, method=experiment)
  evaluate(generations, experiment)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

greedy
perplexity = 2.08
fluency = 0.78
diversity = 0.01, 0.02, 0.03



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

temperature
perplexity = 15.47
fluency = 0.64
diversity = 0.31, 0.78, 0.96



  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

topp
perplexity = 12.27
fluency = 0.72
diversity = 0.28, 0.77, 0.96



## Discussion
- Q2.1: In greedy decoding, what do you observe when generating 10 times from the test prompt?

Ans: all 10 generated responses are the same

- Q2.2: In vanilla sampling, what do you observe when generating 10 times from the test prompt?

Ans: none of the 10 generated responses are the same

- Q2.3: In temperature sampling, play around with the value of temperature $t$. Which value of $t$ makes it equivalent to greedy decoding? Which value of $t$ makes it equivalent to vanilla sampling?

Ans: Greedy: t=0.001; Vanilla: t=1

- Q2.4: In top-$p$ sampling, play around with the value of $p$. Which value of $p$ makes it equivalent to greedy decoding? Which value of $p$ makes it equivalent to vanilla sampling?

Ans: Greedy: p=0; Vanilla: p=0.99

- Q2.5: Report the evaluation metrics (perplexity, fluency, diversity) of all 4 decoding methods. Which methods have the best and worst perplexity? Fluency? Diversity?

Ans:
greedy
perplexity = 2.08;
fluency = 0.78;
diversity = 0.01, 0.02, 0.03

sample
perplexity = 74.71;
fluency = 0.34;
diversity = 0.42, 0.89, 0.99

temperature
perplexity = 15.47;
fluency = 0.64;
diversity = 0.31, 0.78, 0.96

topp
perplexity = 12.27;
fluency = 0.72;
diversity = 0.28, 0.77, 0.96

Best perplexity: Greedy

Worst perplexity: Sample

Best fluency: Greedy

Worst fluency: Sample

Best diversity: Sample

Worst diversity: Greedy